In [ ]:
import pandas as pd
import os

resource_folder = 'resources/1403-all'

all_data = []

for month in os.listdir(resource_folder):
    month_folder = os.path.join(resource_folder, month)

    if os.path.isdir(month_folder):
        for file_name in os.listdir(month_folder):
            file_path = os.path.join(month_folder, file_name)
            df = pd.read_excel(file_path)
            all_data.append(df)


In [ ]:
combined_df = pd.concat(all_data, ignore_index=True)

combined_df.head()

In [ ]:
combined_df.rename(columns={
    combined_df.columns[0]: "axis code",
    combined_df.columns[1]: "axis name",
    combined_df.columns[2]: "start time",
    combined_df.columns[3]: "end time",
    combined_df.columns[4]: "operating time (minutes)",
    combined_df.columns[5]: "Total number of vehicles",
    combined_df.columns[6]: "number of Class 1 vehicles",
    combined_df.columns[7]: "number of Class 2 vehicles",
    combined_df.columns[8]: "number of Class 3 vehicles",
    combined_df.columns[9]: "number of Class 4 vehicles",
    combined_df.columns[10]: "number of Class 5 vehicles",
    combined_df.columns[11]: "average speed",
    combined_df.columns[12]: "number of speeding violations",
    combined_df.columns[13]: "number of unauthorized distance violations",
    combined_df.columns[14]: "number of unauthorized overtaking violations",
    combined_df.columns[15]: "estimated number"
}, inplace=True)

combined_df = combined_df[combined_df['axis code'] != 'کد محور']

combined_df.head()

In [ ]:
from persiantools.jdatetime import JalaliDateTime
import pandas as pd


def convert_jalali_to_gregorian(df):
    df = df.copy()
    df["start time"] = df["start time"].map(lambda x: JalaliDateTime.strptime(x, "%Y/%m/%d %H:%M:%S").to_gregorian())
    df["start time"] = pd.to_datetime(df["start time"])
    df["date"] = df["start time"].dt.date.astype(str)
    df["start hour"] = df["start time"].dt.hour
    return df



In [ ]:
chunk_size = 100000
num_chunks = len(combined_df) // chunk_size + 1

data = []

chunks = [combined_df[i * chunk_size: (i + 1) * chunk_size] for i in range(num_chunks)]

In [ ]:
processed_chunks = convert_jalali_to_gregorian(chunks[9])

data.append(processed_chunks)

In [ ]:
processed_chunks

In [ ]:
print(len(data))

In [ ]:
processed_data = pd.concat(data, ignore_index=True)

processed_data

In [ ]:
df1 = processed_data.copy()

df1['Total number of vehicles'] = pd.to_numeric(df1['Total number of vehicles'], errors='coerce')

daily_traffic = df1.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

daily_traffic.head()

In [ ]:
N = 30

top_counters = daily_traffic.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df = df1[df1['axis code'].isin(top_counters)]

filtered_df['axis code'].unique()

In [ ]:
hourly_traffic = filtered_df.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis = hourly_traffic.loc[hourly_traffic.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 20
final_selected_counters = peak_hours_per_axis.nlargest(M, 'Total number of vehicles')

In [ ]:
final_selected_counters

In [ ]:
unique_axes = combined_df[['axis code', 'axis name']].drop_duplicates()

unique_axes

In [137]:
cluster_mapping = {
    # 🟢 **South**
    'آزادراه قم - گرمسار (چرمشهر)': 'South',
    'آزادراه گرمسار - قم (چرمشهر)': 'South',
    'آزادراه تهران - قم (مجتمع ياس)': 'South',
    'آزادراه قم - تهران (مجتمع ياس)': 'South',
    'آزادراه تهران - قم (عوارضي تهران)': 'South',
    'آزادراه قم - تهران (عوارضي تهران)': 'South',
    'آزادراه قم - تهران (بين فرودگاه و حسن آباد)': 'South',
    'آزادراه تهران - قم (بين فرودگاه و حسن آباد)': 'South',
    'عوارضي آزادراه قم - تهران (بهشت زهرا)(مجموع گيت ها)': 'South',
    'فرودگاه امام خميني - آزادراه قم، تهران': 'South',
    'آزادراه تهران، قم - فرودگاه امام خميني': 'South',
    'آزادراه تهران، قم - اسلامشهر': 'South',
    'اسلامشهر - آزادراه تهران، قم': 'South',
    'تهران - حسن آباد (جاده قديم قم)': 'South',
    'حسن آباد - تهران (جاده قديم قم)': 'South',
    'جاده قديم تهران - قم (حسن آباد) (متناظر توزين)': 'South',
    'جاده قديم تهران، قم (حسن‌آباد - نعلبندان)': 'South',
    'جاده قديم قم، تهران (نعلبندان - حسن‌آباد)': 'South',
    'رباط\u200cکريم - حسن\u200cآباد (وهن آباد - جاده قديم قم)': 'South',
    'حسن\u200cآباد - رباط\u200cکريم (جاده قديم قم - وهن آباد)': 'South',
    'رباط کريم - شهريار': 'South',
    'شهريار - رباط کريم': 'South',
    'آزادراه تهران - ساوه (عوارضي تهران)': 'South',
    'آزادراه ساوه - تهران (عوارضي تهران)': 'South',
    'آزادراه تهران - ساوه (پرند)': 'South',
    'آزادراه ساوه - تهران (پرند)': 'South',
    'آزادراه تهران - ساوه (تهران-رباط کريم)': 'South',
    'آزادراه ساوه - تهران (رباط کريم-تهران)': 'South',
    'عوارضي آزادراه ساوه - تهران (تهران)(مجموع گيت ها)': 'South',
    'عوارضي آزادراه تهران - ساوه (تهران)(مجموع گيت ها)': 'South',
    'جاده قديم تهران - ساوه (رباط‌کريم - پرند)': 'South',
    'جاده قديم ساوه - تهران (پرند - رباط‌کريم)': 'South',
    'آزادراه چرمشهر - آبيک (چرمشهر)': 'South',
    'آزادراه آبيک -چرمشهر (چرمشهر)': 'South',
    'جاده دسترسي احمدآباد مستوفي(آزادگان - اسلامشهر)': 'South',
    'جاده دسترسي احمدآباد مستوفي(اسلامشهر - آزادگان)': 'South',

    # 🔴 **West**
    'آزادراه تهران - کرج (عوارضي قديم)': 'West',
    'آزادراه کرج - تهران (عوارضي قديم)': 'West',
    'جاده مخصوص تهران - کرج (وردآورد)': 'West',
    'جاده مخصوص کرج - تهران (وردآورد)': 'West',
    'تهران - وردآورد (همت)': 'West',
    'وردآورد - تهران (همت)': 'West',
    'شهريار - آدران': 'West',
    'شهريار-کرج (هفت جوي)': 'West',
    'کرج-شهريار (هفت جوي)': 'West',
    'شهريار - تهران (باغستان)': 'West',
    'شهريار - تهران (شهر قدس)': 'West',
    'تهران - شهريار (باغستان)': 'West',
    'تهران - شهريار (شهر قدس)': 'West',
    'ملارد - صفادشت': 'West',
    'صفادشت - ملارد': 'West',
    'شهريار(بلوار وليعصر) - ملارد': 'West',
    'ملارد - شهريار(بلوار وليعصر)': 'West',
    'آزادراه چرمشهر - آبيک (تقاطع آزادراه تهران - ساوه)': 'West',
    'آزادراه چرمشهر - آبيک (تقاطع اخترآباد)': 'West',
    'آزادراه آبيک - چرمشهر (تقاطع آزادراه تهران - ساوه)': 'West',
    'آزادراه آبيک - چرمشهر(تقاطع اخترآباد)': 'West',

    # 🟣 **North**
    'تهران - امام زاده داوود': 'North',
    'امام زاده داوود - تهران': 'North',
    'تهران - لواسانات (طلائيه)': 'North',
    'تهران - لواسانات (سوهانک)': 'North',
    'تهران - لواسانات (تلو)': 'North',
    'لواسانات - تهران (طلائيه)': 'North',
    'لواسانات - تهران (سوهانک)': 'North',
    'لواسانات - تهران (تلو)': 'North',
    'لواسانات - فشم': 'North',
    'فشم - ميگون': 'North',
    'فشم - لواسانات': 'North',
    'ميگون - فشم': 'North',
    'جاجرود - لتيان': 'North',
    'لتيان - جاجرود': 'North',
    'آزادراه تهران- شمال (قطعه اول - محدوده عوارضي تهران)': 'North',
    'آزادراه شمال - تهران (قطعه اول - محدوده عواضي تهران)': 'North',
    'آزادراه تهران- شمال (قطعه اول - بعد از تونل 12)': 'North',
    'آزادراه شمال - تهران (قطعه اول - بعد از تونل 12)': 'North',
    'جاجرود - تهران': 'North',
    'تهران - جاجرود (متناظر توزين)': 'North',

    # 🔵 **East**
    'آزادراه تهران - پرديس (عوارضي پرديس)': 'East',
    'آزادراه پرديس - تهران (عوارضي پرديس)': 'East',
    'آزادراه تهران - پرديس (بومهن)': 'East',
    'آزادراه پرديس - تهران (بومهن)': 'East',
    'عوارضي آزادراه تهران- پرديس (فاز8 ) (مجموع گيت ها)': 'East',
    'عوارضي آزادراه پرديس- تهران (فاز11) (مجموع گيت ها)': 'East',
    'رودهن - دماوند': 'East',
    'دماوند - رودهن': 'East',
    'رودهن - آبعلي': 'East',
    'آبعلي - رودهن': 'East',
    'پلور - آبعلي (سه‌راهي مشاء - آبعلي)': 'East',
    'آبعلي - پلور (آبعلي - سه‌راهي مشاء)': 'East',
    'پلور - آبعلي (امامزاده هاشم - سه‌راهي مشاء)': 'East',
    'آبعلي - پلور (سه‌راهي مشاء - امامزاده هاشم)': 'East',
    'دماوند - فيروزکوه (امين‌آباد - سه‌راه ارجمند)': 'East',
    'فيروزکوه - دماوند (سه‌راه ارجمند - امين‌آباد)': 'East',
    'دماوند - فيروزکوه (گيلاوند - آبسرد)': 'East',
    'فيروزکوه - دماوند (آبسرد - گيلاوند)': 'East',
    'دماوند - فيروزکوه (سربندان - امين‌آباد)': 'East',
    'فيروزکوه - دماوند (امين‌آباد - سربندان)': 'East',
    'فيروزکوه - دماوند (آبسرد - دماوند)': 'East',
    'دماوند - فيروزکوه (دماوند - آبسرد)': 'East',
    'دماوند - فيروزکوه (آبسرد - سربندان)': 'East',
    'فيروزکوه - دماوند (سربندان - آبسرد)': 'East',
    'دماوند - فيروزکوه (سه\u200cراهي ارجمند - فيروزکوه)': 'East',
    'فيروزکوه - دماوند (فيروزکوه - سه\u200cراهي ارجمند)': 'East',
    'فيروزکوه - گدوک': 'East',
    'گدوک - فيروزکوه': 'East',
    'آبسرد - ايوانکي (آبسرد - دوآب)': 'East',
    'ايوانکي - آبسرد (دوآب - آبسرد)': 'East',
    'کمربندي جنوبي رودهن (پرديس - دماوند)': 'East',
    'کمربندي جنوبي رودهن (دماوند - پرديس)': 'East',
    'فيروزکوه - سمنان (فيروزکوه)': 'East',
    'سمنان - فيروزکوه (فيروزکوه)': 'East',

    # 🔵 **Southeast**
    'ري - قرچک': 'Southeast',
    'قرچک - ري': 'Southeast',
    'تهران - پاکدشت': 'Southeast',
    'پاکدشت - تهران': 'Southeast',
    'پاکدشت - شريف آباد': 'Southeast',
    'شريف آباد - پاکدشت': 'Southeast',
    'شريف آباد - گرمسار': 'Southeast',
    'گرمسار - شريف آباد': 'Southeast',
    'پيشوا - شريف‌آباد': 'Southeast',
    'شريف‌آباد - پيشوا': 'Southeast',
    'ورامين - چرمشهر': 'Southeast',
    'چرمشهر - ورامين': 'Southeast',
    'کمربندي دوم تهران (جاده ورامين - شورآباد)': 'Southeast',
    'کمربندي دوم تهران (شورآباد - جاده ورامين)': 'Southeast'
}

print(len(cluster_mapping))


125


In [147]:
cluster_to_axes = {}

for axis_name, cluster in cluster_mapping.items():
    axis_code = unique_axes.loc[unique_axes['axis name'] == axis_name, 'axis code'].values[0]

    if cluster not in cluster_to_axes:
        cluster_to_axes[cluster] = []

    cluster_to_axes[cluster].append(axis_code)


In [149]:
cluster_1 = processed_data[processed_data['axis code'].isin(cluster_to_axes['North'])]

cluster_2 = processed_data[processed_data['axis code'].isin(cluster_to_axes['East'])]

cluster_3 = processed_data[processed_data['axis code'].isin(cluster_to_axes['South'])]

cluster_4 = processed_data[processed_data['axis code'].isin(cluster_to_axes['West'])]

cluster_5 = processed_data[processed_data['axis code'].isin(cluster_to_axes['Southeast'])]



In [165]:
cluster_1 = cluster_1.copy()
cluster_1['Total number of vehicles'] = pd.to_numeric(cluster_1['Total number of vehicles'], errors='coerce')

daily_traffic_1 = cluster_1.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

N = 20

top_counters_1 = daily_traffic_1.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df_1 = cluster_1[cluster_1['axis code'].isin(top_counters_1)]

filtered_df_1['axis code'].unique()

hourly_traffic_1 = filtered_df_1.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis_1 = hourly_traffic_1.loc[hourly_traffic_1.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 10
final_selected_counters_1 = peak_hours_per_axis_1.nlargest(M, 'Total number of vehicles')

final_selected_counters_1

,axis code,start hour,Total number of vehicles
329,113813,17,3447.693452
449,113863,17,2843.353312
44,113213,20,1553.183230
137,113263,17,1475.524390
186,113299,18,980.651786
377,113851,17,959.036424
198,113341,6,904.163690
152,113264,8,870.269625
251,113801,11,833.867987
78,113249,6,831.420896


In [164]:
cluster_2 = cluster_2.copy()
cluster_2['Total number of vehicles'] = pd.to_numeric(cluster_2['Total number of vehicles'], errors='coerce')

daily_traffic_2 = cluster_2.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

N = 20

top_counters_2 = daily_traffic_2.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df_2 = cluster_2[cluster_2['axis code'].isin(top_counters_2)]

filtered_df_2['axis code'].unique()

hourly_traffic_2 = filtered_df_2.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis_2 = hourly_traffic_2.loc[hourly_traffic_2.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 10
final_selected_counters_2 = peak_hours_per_axis_2.nlargest(M, 'Total number of vehicles')

final_selected_counters_2

,axis code,start hour,Total number of vehicles
89,113257,17,2786.798450
329,113464,17,2750.883333
65,113256,17,2630.775385
161,113414,17,2602.422222
17,113206,17,2463.089231
401,113469,17,2461.779762
281,113453,17,2433.616314
107,113403,11,2374.296636
35,113207,11,2245.063253
227,113419,11,2242.274627


In [163]:
cluster_3 = cluster_3.copy()
cluster_3['Total number of vehicles'] = pd.to_numeric(cluster_3['Total number of vehicles'], errors='coerce')

daily_traffic_3 = cluster_3.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

N = 40

top_counters_3 = daily_traffic_3.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df_3 = cluster_3[cluster_3['axis code'].isin(top_counters_3)]

filtered_df_3['axis code'].unique()

hourly_traffic_3 = filtered_df_3.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis_3 = hourly_traffic_3.loc[hourly_traffic_3.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 20
final_selected_counters_3 = peak_hours_per_axis_3.nlargest(M, 'Total number of vehicles')

final_selected_counters_3

,axis code,start hour,Total number of vehicles
703,114110,7,5219.806228
497,113612,17,5173.779264
305,113260,17,4908.359756
66,113204,18,4877.212121
258,113254,18,4770.806122
233,113253,17,4631.532738
463,113604,7,4455.977564
569,113654,17,4375.754601
808,114160,16,4225.343972
42,113203,18,4148.215569


In [162]:
cluster_4 = cluster_4.copy()
cluster_4['Total number of vehicles'] = pd.to_numeric(cluster_4['Total number of vehicles'], errors='coerce')

daily_traffic_4 = cluster_4.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

N = 20

top_counters_4 = daily_traffic_4.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df_4 = cluster_4[cluster_4['axis code'].isin(top_counters_4)]

filtered_df_4['axis code'].unique()

hourly_traffic_4 = filtered_df_4.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis_4 = hourly_traffic_4.loc[hourly_traffic_4.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 10
final_selected_counters_4 = peak_hours_per_axis_4.nlargest(M, 'Total number of vehicles')

final_selected_counters_4

,axis code,start hour,Total number of vehicles
448,114501,16,6747.357798
471,114551,15,6432.787879
17,113202,17,4277.660606
159,113904,15,4150.406452
90,113428,18,4071.348243
42,113252,18,4042.628399
257,113954,17,3760.527508
281,113959,17,3272.365314
137,113903,17,2853.513932
175,113909,7,2729.543071


In [161]:
cluster_5 = cluster_5.copy()
cluster_5['Total number of vehicles'] = pd.to_numeric(cluster_5['Total number of vehicles'], errors='coerce')

daily_traffic_5 = cluster_5.groupby(['axis code', 'date'])['Total number of vehicles'].sum().reset_index()

N = 20

top_counters_5= daily_traffic_5.groupby('axis code')['Total number of vehicles'].mean().nlargest(N).index

filtered_df_5 = cluster_5[cluster_5['axis code'].isin(top_counters_5)]

filtered_df_5['axis code'].unique()

hourly_traffic_5 = filtered_df_5.groupby(['axis code', 'start hour'])['Total number of vehicles'].mean().reset_index()

peak_hours_per_axis_5 = hourly_traffic_5.loc[hourly_traffic_5.groupby('axis code')['Total number of vehicles'].idxmax()]

M = 10
final_selected_counters_5 = peak_hours_per_axis_5.nlargest(M, 'Total number of vehicles')

final_selected_counters_5

,axis code,start hour,Total number of vehicles
41,113259,17,4615.584416
17,113209,17,4069.775735
90,114302,18,3770.380062
281,114451,17,3583.321429
199,114401,7,3414.576119
151,114352,7,3317.931373
305,114463,17,1867.328467
223,114413,7,1486.957143
246,114418,6,1383.672000
331,114468,19,1156.105882
